In [1]:
!pip install -r requirements.txt

In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from public_tests import *

%matplotlib inline 
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [3]:
# Zero padding

def zero_pad(X, pad):
    X_pad = np.pad(X, ((0,0),(pad,pad),(pad,pad),(0,0)), mode='constant', constant_values=(0,0))
    return X_pad

In [4]:
# Single step of convolution

def conv_single_step(a_slice_prev, W, b):
    s = np.multiply(a_slice_prev, W)
    z = np.sum(s)
    z = z + float(b)
    return z

In [5]:
# Convolutional neural networks - forward pass

def conv_forward(A_prev, W, b, hparameters):
    (m, n_H_prev, n_W_prev, n_C_prev) = (A_prev.shape[0], A_prev.shape[1], A_prev.shape[2], A_prev.shape[3])
    (f, f, n_C_prev, n_C) = (W.shape[0], W.shape[1], W.shape[2], W.shape[3])

    stride = hparameters['stride']
    pad = hparameters['pad']
    
    n_H = int((n_H_prev + 2*pad - f)/stride) + 1
    n_W = int((n_W_prev + 2*pad - f)/stride) + 1

    z = np.zeros((m, n_H, n_W, n_C))

    A_prev_pad = zero_pad(A_prev, pad)

    for i in range(m):
        a_prev_pad = A_prev_pad[i]
        
        for h in range(n_H):
            vert_start = stride * h
            vert_end = vert_start + f
            
            for w in range(n_W):
                horiz_start = stride * w
                horiz_end = horiz_start + f
                
                for c in range(n_C):
                    a_slice_prev = A_prev_pad[i, vert_start:vert_end, horiz_start:horiz_end, :]
                    weights = W[:,:,:,c]
                    biases = b[:,:,:,c]
                    z[i,h,w,c] = conv_single_step(a_slice_prev, weights, biases)
    
    cache = (A_prev, W, b,hparameters)
    return z, cache
                

In [11]:
# Forward pooling

def pool_forward(A_prev, hparameters, mode = 'max'):
    (m, n_H_prev, n_W_prev, n_C_prev) = (A_prev.shape)

    f = hparameters['f']
    stride = hparameters['stride']

    n_H = int(1 + (n_H_prev - f)/stride)
    n_W = int(1 + (n_W_prev - f)/stride)
    n_C = n_C_prev

    A = np.zeros((m, n_H, n_W, n_C))

    for i in range(m):
        
        for h in range(n_H):
            vert_start = stride * h
            vert_end = vert_start + f

            for w in range(n_W):
                horiz_start = stride * w
                horiz_end = horiz_start + f

                for c in range(n_C):
                    a_prev_slice = A_prev[i]

                    if mode == 'max':
                        A[i,h,w,c] = np.max(a_prev_slice[vert_start:vert_end, horiz_start:horiz_end, c])
                    elif mode == 'average':
                        A[i,h,w,c] = np.mean(a_prev_slice[vert_start:vert_end, horiz_start:horiz_end,c])

    cache = (A_prev, hparameters)

    return A, cache
            
